In [1]:
import numpy as np
import argparse
import importlib
import torch
import os
from torch.utils.data.sampler import SubsetRandomSampler

import torchvision
import torchvision.transforms as transforms
from torchvision import datasets

from src.utils.worker_utils import read_data
from config import OPTIMIZERS, DATASETS, MODEL_PARAMS, TRAINERS

from data.data_loader import GetLoader

import warnings

warnings.filterwarnings('ignore')

def read_options():
    parser = argparse.ArgumentParser()

    parser.add_argument('--algo',
                        help='name of trainer;',
                        type=str,
                        choices=OPTIMIZERS,
                        default='fedavg4')
    parser.add_argument('--dataset',
                        help='name of dataset;',
                        type=str,
                        default='mnist_all_data_1_random_niid')
                        # default='svhn_all_data_1_random_niid')
    parser.add_argument('--finetune_dataset',
                        help='name of finetune dataset;',
                        type=str,
                        default='mnist-m')
#                         default = 'mnist')
    parser.add_argument('--finetune_lr',
                        help='lr for finetune;',
                        type=float,
                        default=0.01)
    parser.add_argument('--finetune_wd',
                        help='weight decay for finetune;',
                        type=float,
                        default=0.0)
    parser.add_argument('--finetune_epochs',
                        help='epochs for finetune;',
                        type=int,
                        default=100)
#                         default = 2)
    parser.add_argument('--model',
                        help='name of model;',
                        type=str,
                        default='LeNet')
                        # default = 'cnn')
    parser.add_argument('--wd',
                        help='weight decay parameter;',
                        type=float,
                        default=0.0)
    parser.add_argument('--gpu',
                        action='store_true',
                        default=True,
                        help='use gpu (default: False)')
    parser.add_argument('--noprint',
                        action='store_true',
                        default=False,
                        help='whether to print inner result (default: False)')
    parser.add_argument('--noaverage',
                        action='store_true',
                        default=False,
                        help='whether to only average local solutions (default: True)')
    parser.add_argument('--device',
                        help='selected CUDA device',
                        default=0,
                        type=int)
    parser.add_argument('--num_round',
                        help='number of rounds to simulate;',
                        type=int,
                        # default=1000)
                        default = 200)
                        
    parser.add_argument('--eval_every',
                        help='evaluate every ____ rounds;',
                        type=int,
                        default=5)
                        
    parser.add_argument('--clients_per_round',
                        help='number of clients trained per round;',
                        type=int,
                        default=10)
                       
    parser.add_argument('--batch_size',
                        help='batch size when clients train on data;',
                        type=int,
                        default=64)
    parser.add_argument('--ft_batch_size',
                        help='batch size when clients train on data;',
                        type=int,
                        default=1024)
    parser.add_argument('--num_epoch',
                        help='number of epochs when clients train on data;',
                        type=int,
                        default=10)
                        # default =1)
    
    parser.add_argument('--lr',
                        help='learning rate for inner solver;',
                        type=float,
                        default=0.01)
    parser.add_argument('--seed',
                        help='seed for randomness;',
                        type=int,
                        default=0)
    parser.add_argument('--dis',
                        help='add more information;',
                        type=str,
                        default='')
    parsed = parser.parse_args([])
    options = parsed.__dict__
    options['gpu'] = options['gpu'] and torch.cuda.is_available()

    # Set seeds
    np.random.seed(1 + options['seed'])
    torch.manual_seed(12 + options['seed'])
    if options['gpu']:
        torch.cuda.manual_seed_all(123 + options['seed'])

    # read data
    idx = options['dataset'].find("_")
    if idx != -1:
        dataset_name, sub_data = options['dataset'][:idx], options['dataset'][idx+1:]
    else:
        dataset_name, sub_data = options['dataset'], None
    assert dataset_name in DATASETS, "{} not in dataset {}!".format(dataset_name, DATASETS)

    # Add model arguments
    options.update(MODEL_PARAMS(dataset_name, options['model']))

    # Load selected trainer
    trainer_path = 'src.trainers.%s' % options['algo']
    mod = importlib.import_module(trainer_path)
    trainer_class = getattr(mod, TRAINERS[options['algo']])

    # Print arguments and return
    max_length = max([len(key) for key in options.keys()])
    fmt_string = '\t%' + str(max_length) + 's : %s'
    print('>>> Arguments:')
    for keyPair in sorted(options.items()):
        print(fmt_string % keyPair)

    return options, trainer_class, dataset_name, sub_data


# Parse command line arguments
options, trainer_class, dataset_name, sub_data = read_options()
options["dataset_name"] = dataset_name

train_path = os.path.join('./data', dataset_name, 'data', 'train')
test_path = os.path.join('./data', dataset_name, 'data', 'test')

# `dataset` is a tuple like (cids, groups, train_data, test_data)
all_data_info = read_data(train_path, test_path, dataset_name, sub_data)

# # Call appropriate trainer
# trainer = trainer_class(options, all_data_info)
# trainer.train()

# # FL training finish here, save the latest server model
# flat_model_param = trainer.latest_model
# PATH = f"./models/{options['model']}_{dataset_name}_{options['algo']}_{options['seed']}"
# torch.save(flat_model_param, PATH)



from src.utils.torch_utils import get_flat_grad, get_state_dict, get_flat_params_from, set_flat_params_to
from src.models.model import choose_model
from ft_functions import *

# # This is the final flattened model params
# flat_model_params = trainer.latest_model

# Optional: load from the saved model.
PATH = f"./models/{options['model']}_{dataset_name}_{options['algo']}_{options['seed']}"
flat_model_params = torch.load(PATH)


# Initialize new models
model_source_only = choose_model(options) # baseline: lower bound
model_ft = choose_model(options) # baseline: standard fine-tune
model_target_only = choose_model(options) # baseline: upper bound
model_flag = choose_model(options) # test if the model archetecture is proper for target domain
# If performances of model_flag and model_target_only deviates, then the model is proper for target domain.

# Assign model params
set_flat_params_to(model_source_only, flat_model_params)
set_flat_params_to(model_ft, flat_model_params)
# Now model is set with flat_model_params

# Start fine-tuning below
# First, freeze all but last layer
def freeze(model):
    for mod in model.children():
        for params in mod.parameters():
            params.requires_grad = False

    for params in mod.parameters():
        params.requires_grad = True

freeze(model_flag)
freeze(model_ft)


# # Next fine-tune the model on mnist:
# device = torch.device(f"cuda:{options['device']}")
device = torch.device('cpu')
criterion = torch.nn.CrossEntropyLoss()
kwargs = {'num_workers': 32}




# Prepare data
# assert options['finetune_dataset'] == 'mnist-m' # now we only support transfer from svhn to mnist.
if options['finetune_dataset'] == "mnist-m":
    target_image_root = './data/TARGET/mnist-m/mnist_m'

    tg_train_list = os.path.join(target_image_root, 'mnist_m_train_labels.txt')
    tg_test_list = os.path.join(target_image_root, 'mnist_m_test_labels.txt')
    
    img_transform_target = transforms.Compose([
    transforms.Resize(28),
    transforms.ToTensor(),
    lambda x: x*255,
    transforms.Grayscale()
    ])

    tg_trainset = GetLoader(
        data_root=os.path.join(target_image_root, 'mnist_m_train'),
        data_list=tg_train_list,
        transform=img_transform_target
    )

    tg_testset = GetLoader(
        data_root=os.path.join(target_image_root, 'mnist_m_test'),
        data_list=tg_test_list,
        transform=img_transform_target
    )

    tg_train_loader = torch.utils.data.DataLoader(
        dataset=tg_trainset,
        batch_size=options['ft_batch_size'],
        shuffle=True,
        pin_memory=True,
        **kwargs)

    tg_test_loader = torch.utils.data.DataLoader(
        dataset=tg_testset,
        batch_size=options['ft_batch_size'],
        shuffle=True,
        pin_memory=True,
        **kwargs)

elif options['finetune_dataset'] == "mnist":
    data_path = './data/TARGET/mnist'
    
    img_transform_target = transforms.Compose([
    transforms.ToTensor(),
    lambda x: x*255,
    ])
    
    trainset = torchvision.datasets.MNIST(root=data_path, train =True, download=False, transform=img_transform_target)
    testset = torchvision.datasets.MNIST(root=data_path, train =False, download=False, transform=img_transform_target)
    tg_train_loader = torch.utils.data.DataLoader(trainset, batch_size=options['ft_batch_size'], shuffle=True,pin_memory=True, **kwargs)
    tg_test_loader = torch.utils.data.DataLoader(testset, batch_size=options['ft_batch_size'], shuffle=True,pin_memory=True, **kwargs)

/home/ichien3/miniconda3/envs/FLTransfer/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


>>> Arguments:
	             algo : fedavg4
	       batch_size : 64
	clients_per_round : 10
	          dataset : mnist_all_data_1_random_niid
	           device : 0
	              dis : 
	       eval_every : 5
	 finetune_dataset : mnist-m
	  finetune_epochs : 100
	      finetune_lr : 0.01
	      finetune_wd : 0.0
	    ft_batch_size : 1024
	              gpu : True
	      input_shape : (1, 28, 28)
	               lr : 0.01
	            model : LeNet
	        noaverage : False
	          noprint : False
	        num_class : 10
	        num_epoch : 10
	        num_round : 200
	             seed : 0
	               wd : 0.0
>>> Read data from:
     ./data/mnist/data/train/all_data_1_random_niid.pkl
     ./data/mnist/data/test/all_data_1_random_niid.pkl


In [2]:
print(model_target_only)

LeNet_MNIST(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


# Use functorch to get Jacobian and see if it is compatible with loss.backward

In [40]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from functools import partial
from functorch import make_functional_with_buffers, vmap, grad, jacfwd, jacrev


# This is for computing loss with fmodel.
def compute_loss_stateless_model(fmodel, criterion, buffers, sample, target,params):
#     # Below two lines are only need if batchsize=1
#     batch = sample.unsqueeze(0)
#     targets = target.unsqueeze(0)
    
    predictions = fmodel(params, buffers, sample)
    loss = criterion(predictions, target)
    return loss

def assign_params(model,params):
    model.zero_grad() # prevent aggregate gradient.
    for p_model, p_fmodel in zip(model.parameters(),params):
        p_model.grad = p_fmodel.grad
        p_model.data = p_fmodel.data
    return None

In [41]:
for sample, target in tg_train_loader:
    
    # setup stateless model
    fmodel, params, buffers = make_functional_with_buffers(model_target_only)
    
    # compute loss. it is now a fuction of params that will return loss.
    compute_loss_stateless_model_partial = partial(compute_loss_stateless_model,fmodel,
                                                   criterion, buffers, sample, target)
    # compute jacobian with input: params
    jacobian = jacrev(compute_loss_stateless_model_partial)(params)
    
    # In this example, we assume below operation is f(). You should replace it with your method.
    temp = 0
    for j in jacobian:
        temp += j.view(-1).sum()
    
    # compute loss with params, sample, and target.
    predictions = fmodel(params, buffers, sample)
    loss = criterion(predictions, target) + temp

    # the result is the per-batch jacobian. You can do whatever you want can 
    # (i.e., loss += f(jacobian)) and call loss.backward()
    loss.backward()
    
    # Note that now the gradient information are updated in "params"
    # You have to put params back to the model
    assign_params(model_target_only,params)
    
    # Then you can call the optimizer.
    break

## Some notes

Note that the key idea of functorch and stateless model is to make fmodel as a pure function, with input (params, sample). I think you should use fmodel and params to compute the grad (w.r.t. your new loss) first and then update the resulting params to original model (via function assign_params) before calling the optimizer.